코드 참고 및 출처

https://velog.io/@hwanython/Movielens-Recommendation-%EB%AA%A8%EB%8D%B8-%ED%95%B4%EB%B3%B4%EA%B8%

https://github.com/pgvector/pgvector

### Library Import

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import psycopg2
from implicit.als import AlternatingLeastSquares
import os
from scipy.sparse import csr_matrix
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings(action='ignore')

# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Preparation

In [7]:
# 데이터 준비
path = './'
ratings = pd.read_csv(path + 'ratings.csv', low_memory=False, sep = '|')
movies = pd.read_csv(path + 'movies.csv', low_memory=False, sep = '|')

ratings = pd.merge(ratings, movies[['id', 'title']], how='left', left_on='movie_id', right_on='id')
ratings = ratings[ratings['title'].notnull()].reset_index(drop=True)

# 태그 데이터 로드
tags = pd.read_csv('tags.csv', sep = '|')

# 영화별 태그를 하나의 문자열로 결합
tags_grouped = tags.groupby('id')['tag'].apply(lambda x: ' '.join(x)).reset_index()

        id               tag
0        1             funny
1        2   Highly quotable
2        3      will ferrell
3        4      Boxing story
4        5               MMA
...    ...               ...
3678  3679         for katie
3679  3680           austere
3680  3681            gun fu
3681  3682  heroic bloodshed
3682  3683  Heroic Bloodshed

[3683 rows x 2 columns]


### Vectorization

In [4]:
### als 기반 유저 벡터와 아이템 벡터 생성
# rating 3 이상만 사용
filtered_ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(filtered_ratings)

print(f'orginal_data_size: {len(ratings)}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / len(ratings):.2%}')

filtered_ratings.rename(columns={'rating':'count'}, inplace=True)
filtered_ratings.drop(columns = ['timestamp', 'title'], axis =1 , inplace = True)

movies_unique = movies['title'].unique()
title_to_idx = {k:v for k,v in enumerate(movies_unique)}

num_user = filtered_ratings['user_id'].nunique()
num_movie = filtered_ratings['movie_id'].nunique()
print(num_user)
print(num_movie)
print(len(filtered_ratings))

filtered_ratings=filtered_ratings.astype('int64')

csr_data = csr_matrix((filtered_ratings["count"], (filtered_ratings.user_id, filtered_ratings.movie_id)))

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose.)
csr_data_transpose = csr_data.T
csr_data_transpose

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# 모델 훈련
als_model.fit(csr_data_transpose)

# 사용자-아이템 상호작용 행렬을 분해하여 숨겨진 요인(latent factors)을 학습 -> 사용자와 아이템 벡터의 내적을 통해 추천
# name_vector, movie_vector = als_model.user_factors[1], als_model.item_factors[610]
# np.dot(name_vector, movie_vector)


## 태그 벡터 생성
# TF-IDF 벡터라이저를 사용해 태그를 벡터화
vectorizer = TfidfVectorizer(stop_words='english')
tag_vectors = vectorizer.fit_transform(tags_grouped['tag'])

# 태그 벡터를 리스트로 변환
tag_vectors_list = tag_vectors.toarray().tolist()

orginal_data_size: 100836, filtered_data_size: 81763
Ratio of Remaining Data is 81.09%
609
8452
81763


100%|██████████| 15/15 [00:01<00:00,  7.66it/s]


### DB INSERT

In [6]:
## DB INSERT
database = 
user = 
password = 
host = 
port = 
data_path = "./"

# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(
        database=database, user=user, password=password, host=host, port=port
    )
cur = conn.cursor()

# #install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
conn.commit()


# Register the vector type with psycopg2
register_vector(conn)

# 영화 임베딩 테이블 생성
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS movie_embeddings (
#         movie_id INTEGER UNIQUE NOT NULL,
#         title VARCHAR,
#         movie_embedding VECTOR(100),
#         FOREIGN KEY (movie_id) REFERENCES movies(id)
#     );
# """)
# Alter table로 변경
cur.execute("""
    ALTER TABLE movies ADD COLUMN IF NOT EXISTS movie_embedding VECTOR(100);
""")
conn.commit()

# 영화 임베딩 데이터 준비
movie_titles = {int(row['id']): row['title'] for _, row in movies.iterrows()}
data_list = [(movieid, movie_titles.get(int(movieid), "Unknown"), als_model.item_factors[movieid].tolist())
             for movieid in range(len(als_model.item_factors))]
data_list.pop(0)

# 데이터베이스에 배치 삽입
execute_values(cur, "INSERT INTO movies (id, title, movie_embedding) VALUES %s ON CONFLICT (id) DO UPDATE SET movie_embedding = EXCLUDED.movie_embedding", data_list)
conn.commit()

# 불필요한 영화 데이터 전부 삭제
cur.execute("""
    DELETE FROM ratings WHERE movie_id IN (SELECT id FROM movies WHERE title = 'Unknown' OR movie_embedding IS NULL);
    DELETE FROM movie_genre WHERE movie_id IN (SELECT id FROM movies WHERE title = 'Unknown' OR movie_embedding IS NULL);
    DELETE FROM tags WHERE movie_id IN (SELECT id FROM movies WHERE title = 'Unknown' OR movie_embedding IS NULL);
    DELETE FROM movies WHERE title = 'Unknown' OR movie_embedding IS NULL;
""")
conn.commit()


# 사용자 임베딩 테이블 생성
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS user_embeddings (
#         user_id INTEGER UNIQUE NOT NULL,
#         user_embedding VECTOR(100),
#         FOREIGN KEY (user_id) REFERENCES users(id)
#     );
# """)
# Alter table로 변경
# cur.execute("""
#     ALTER TABLE users ADD COLUMN IF NOT EXISTS user_embedding VECTOR(100);
# """)
# conn.commit()

# # 사용자 임베딩 데이터 준비
# user_data_list = [(user_id, als_model.user_factors[user_id].tolist())
#                   for user_id in range(len(als_model.user_factors))]
# user_data_list.pop(0)

# # 데이터베이스에 배치 삽입
# execute_values(cur, "INSERT INTO users (id, user_embedding) VALUES %s ON CONFLICT (id) DO UPDATE SET user_embedding = EXCLUDED.user_embedding", user_data_list)
# conn.commit()

# 태그 임베딩 테이블 생성
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS tag_embeddings (
#         movie_id INTEGER PRIMARY KEY,
#         tag_embedding VECTOR(%s),
#         FOREIGN KEY (movie_id) REFERENCES movies(id),
#     );
# """, (len(tag_vectors_list[0]),))
# Alter table로 변경
# cur.execute("""
#     ALTER TABLE movies ADD COLUMN IF NOT EXISTS tag_embedding VECTOR(%s);
# """, (len(tag_vectors_list[0]),))
# conn.commit()

# # 태그 임베딩 데이터 준비
# data_list = [(int(tags_grouped['movie_id'][i]), tag_vectors_list[i]) for i in range(len(tag_vectors_list))]
# data_list.pop(0)

# # 데이터베이스에 배치 삽입
# execute_values(cur, "INSERT INTO movies (id, tag_embedding) VALUES %s ON CONFLICT (id) DO UPDATE SET tag_embedding = EXCLUDED.tag_embedding", data_list)
# conn.commit()

# # 불필요한 영화 데이터 전부 삭제
# cur.execute("""
#     DELETE FROM ratings WHERE movie_id IN (SELECT id FROM movies WHERE title = 'Unknown' OR tag_embedding IS NULL);
#     DELETE FROM movie_genre WHERE movie_id IN (SELECT id FROM movies WHERE title = 'Unknown' OR tag_embedding IS NULL);
#     DELETE FROM tags WHERE movie_id IN (SELECT id FROM movies WHERE title = 'Unknown' OR tag_embedding IS NULL);
#     DELETE FROM movies WHERE title = 'Unknown' OR tag_embedding IS NULL;
# """)
# conn.commit()


# # 연결 종료
cur.close()
conn.close()

        id               tag
0        1             funny
1        2   Highly quotable
2        3      will ferrell
3        4      Boxing story
4        5               MMA
...    ...               ...
3678  3679         for katie
3679  3680           austere
3680  3681            gun fu
3681  3682  heroic bloodshed
3682  3683  Heroic Bloodshed

[3683 rows x 2 columns]


KeyError: 'movie_id'

### TEST CODE

In [ ]:
# 사용자별로 선호할 만한 영화 추천
import psycopg2
import numpy as np
from scipy.spatial.distance import cosine

database = 
user = 
password = 
host = 
port = 
data_path = "./"

# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(
        database=database, user=user, password=password, host=host, port=port
    )
cur = conn.cursor()

# 사용자 임베딩 데이터 가져오기
def get_user_embedding(user_id):
    cur.execute("SELECT user_embedding FROM user_embeddings WHERE userId = %s", [user_id])
    embedding = cur.fetchone()[0]
    return np.array(embedding)

# 영화 임베딩 데이터 가져오기
def get_movie_embeddings():
    cur.execute("SELECT movie_id, title, movie_embedding FROM movie_embeddings")
    all_movies = cur.fetchall()
    movie_embeddings = {movieid: (title, np.array(movie_embedding)) for movieid, title, movie_embedding in all_movies}
    return movie_embeddings

# 유사한 영화 추천 함수
def recommend_movies(user_id, top_n=5):
    user_embedding = get_user_embedding(user_id)
    movie_embeddings = get_movie_embeddings()
    
    scores = {movieid: (title, np.dot(user_embedding, movie_embedding)) for movieid, (title, movie_embedding) in movie_embeddings.items()}
    recommended_movies = sorted(scores.items(), key=lambda x: x[1][1], reverse=True)[:top_n]
    
    return recommended_movies

# 예시: 사용자 ID가 1인 사용자를 위한 영화 추천
recommended_movies = recommend_movies(1)
for movie in recommended_movies:
    print(f"Movie ID: {movie[0]}, Title: {movie[1][0]}, Score: {movie[1][1]:.4f}")


# 연결 종료
cur.close()
conn.close()

: 

In [ ]:
## 영화 아이디로 검색 시, 유사한 tag 기반으로 영화 추천
import psycopg2
import numpy as np
from scipy.spatial.distance import cosine

database = 
user = 
password = 
host = 
port = 
data_path = "./"

# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(
        database=database, user=user, password=password, host=host, port=port
    )
cur = conn.cursor()


# 영화 임베딩 데이터 가져오기
def get_tag_embedding(movieid):
    cur.execute("SELECT tag_embedding FROM tag_embeddings WHERE movie_id = %s", [movieid])
    result = cur.fetchone()
    if result:
        embedding = result[0]
        return np.array(embedding)
    return None

# 유사한 영화 추천 함수
def recommend_similar_movies(movieid, top_n=5):
    target_embedding = get_tag_embedding(movieid)
    if target_embedding is None:
        print(f"No embedding found for movieid: {movieid}")
        return []
    
    cur.execute("""
        SELECT te.movie_id, m.title, te.tag_embedding
        FROM tag_embedding te
        JOIN movies m ON te.movie_id = m.id
        WHERE te.movie_id != %s
    """, [movieid])
    all_movies = cur.fetchall()
    
    similarities = []
    for movie in all_movies:
        other_movieid, title, embedding = movie
        if embedding is not None:
            embedding = np.array(embedding)
            similarity = 1 - cosine(target_embedding, embedding)
            similarities.append((other_movieid, title, similarity))
    
    # 유사도가 높은 순으로 정렬하여 상위 N개 반환
    similar_movies = sorted(similarities, key=lambda x: x[2], reverse=True)[:top_n]
    return similar_movies

# 예시: 영화 ID가 50인 영화와 유사한 영화 추천
similar_movies = recommend_similar_movies(50)
for movie in similar_movies:
    print(f"Movie ID: {movie[0]}, Title: {movie[1]}, Similarity: {movie[2]:.4f}")

# 연결 종료
cur.close()
conn.close()

: 

In [ ]:
## 태그 임베팅 벡터 기반 유사한 영화 검색
import psycopg2
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

database = 
user = 
password = 
host = 
port = 
data_path = "./"

# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(
        database=database, user=user, password=password, host=host, port=port
    )
cur = conn.cursor()

# 영화 태그 임베딩 데이터 가져오기
def get_all_movie_embeddings():
    cur.execute("""
        SELECT te.movie_id, m.title, te.tag_embedding
        FROM tag_embeddings te
        JOIN movies m ON te.movie_id = m.movie_id
    """)
    return cur.fetchall()

# 태그 임베딩 벡터화 함수
def vectorize_tags(tag_list):
    vectorizer = TfidfVectorizer(stop_words='english')
    tag_vectors = vectorizer.fit_transform(tag_list)
    return tag_vectors, vectorizer

# 사용자 입력 태그를 벡터화하여 유사한 영화 추천 함수
def recommend_movies_by_tags(input_tags, top_n=5):
    all_movies = get_all_movie_embeddings()
    
    # 태그 임베딩 벡터와 제목을 분리
    movieids, titles, embeddings = zip(*[(movieid, title, " ".join(map(str, embedding))) for movieid, title, embedding in all_movies])
    
    # 태그 벡터화
    tag_texts = list(embeddings)
    _, vectorizer = vectorize_tags(tag_texts)
    input_vector = vectorizer.transform([input_tags]).toarray()[0]
    
    similarities = []
    for movieid, title, embedding in zip(movieids, titles, tag_texts):
        embedding_vector = vectorizer.transform([embedding]).toarray()[0]
        similarity = 1 - cosine(input_vector, embedding_vector)
        similarities.append((movieid, title, similarity))
    
    # 유사도가 높은 순으로 정렬하여 상위 N개 반환
    similar_movies = sorted(similarities, key=lambda x: x[2], reverse=True)[:top_n]
    return similar_movies

# 예시: 사용자가 입력한 태그로 유사한 영화 추천
input_tags = "action adventure"
similar_movies = recommend_movies_by_tags(input_tags)
for movie in similar_movies:
    print(f"Movie ID: {movie[0]}, Title: {movie[1]}, Similarity: {movie[2]:.4f}")

# 연결 종료
cur.close()
conn.close()

: 